In [5]:

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import spacy
nlp1 = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
import tkinter as tk
from nltk.tokenize import word_tokenize
ps  = SnowballStemmer("english")
from tkinter import messagebox
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
import pickle


print("starting setup")
ekphrasis_process = TextPreProcessor(
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True, 
    segmenter="twitter", 
    corrector="twitter", 
    unpack_hashtags=True,  
    unpack_contractions=True, 
    spell_correct_elong=False, 
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)

print("finished setup")


def ekphrasis_word(word):

    return(" ".join(ekphrasis_process.pre_process_doc(word)))


def get_ekphrasis(word):
    import re
    b = ekphrasis_word(word)
    TAG_RE = re.compile(r'<[^>]+>')
    res = TAG_RE.sub('',b)
    res = res.rstrip().lstrip()
    return res    

def spacy_lemmatizer(text):
    doc = nlp1(text)
    return (" ".join([token.lemma_ for token in doc]))

ps = PorterStemmer()


starting setup
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...
finished setup


In [6]:
def remove_stopwords(text):
    
    nlp = English()
    my_doc = nlp(text)
    # Create list of word tokens
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    filtered_sentence =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    res = " ".join(x for x in filtered_sentence)   
    return res



In [7]:

if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def textinitialpreprocess(def_string):
    given_string = def_string.lower()
    given_string = cleanHtml(given_string)
    given_string = cleanPunc(given_string)
    given_string = keepAlpha(given_string)
    return given_string



In [13]:
window = tk.Tk()
window.title("SCC Classifier")
window.geometry("1000x150")
copy_s = u"\u00A9" "Manish Puri"
window.configure(background="yellow")
title = tk.Label( text= "This tool is developed to map input tweet or ordinances to SCC", font=("Arial", 10), background="yellow")
title.grid(column=1, row=0) 
label2 = tk.Label( text= "Enter tweet/ordinance", font=("Arial", 10), background="yellow")
label2.grid(column=0, row=3)
label4 = tk.Label( text= copy_s  , font=("Arial", 10), background="yellow")
entry_field1= tk.Entry(width=120, bg = 'light gray', font = 'Calibri', fg = 'green', justify = 'center')
entry_field1.grid(column=1,row=3)
dict_phrases =  {'Smart Economy':0, 'Smart Mobility':0, 'Smart Environment':0, 'Smart Governance':0, 'Smart People':0, 'Smart Living':0, 'No Matches':0}


def phrase_generator():   
    economyWordCount    = 0  
    environmentWordCount= 0
    governanceWordCount = 0
    peopleWordCount     = 0
    mobilityWordCount   = 0 
    livingWordCount     = 0   
    file = open('SCC_KB.pkl', 'rb')
    economy = pickle.load(file)
    mobility = pickle.load(file)
    environment = pickle.load(file)
    governance = pickle.load(file)
    people = pickle.load(file)
    living = pickle.load(file)

    counter = 0
    arrayno=[]
    name = str(entry_field1.get())
    name = textinitialpreprocess(name)
    name_without_stopwords = remove_stopwords(name)

    for el in economy:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1
            economyWordCount = economyWordCount + 1
            dict_phrases['Smart Economy'] +=1
            name_without_stopwords = name_without_stopwords.replace(el,'')            

    for el in mobility:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1            
            mobilityWordCount = mobilityWordCount + 1     
            dict_phrases['Smart Mobility'] +=1  
            name_without_stopwords = name_without_stopwords.replace(el,'')            

    for el in environment:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1
            environmentWordCount = environmentWordCount + 1     
            dict_phrases['Smart Environment'] +=1  
            name_without_stopwords = name_without_stopwords.replace(el,'') 

    for el in governance:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1
            governanceWordCount = governanceWordCount + 1     
            dict_phrases['Smart Governance'] +=1  
            name_without_stopwords = name_without_stopwords.replace(el,'')

    for el in living:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1
            livingWordCount = livingWordCount + 1
            dict_phrases['Smart Living'] +=1   
            name_without_stopwords = name_without_stopwords.replace(el,'')

    for el in people:
        if el in name_without_stopwords or el in get_ekphrasis(name_without_stopwords) or el in spacy_lemmatizer(name_without_stopwords):
            counter+=1
            peopleWordCount = peopleWordCount + 1
            dict_phrases['Smart People'] +=1   
            name_without_stopwords = name_without_stopwords.replace(el,'')  
    
    if counter==0:
        dict_phrases['No Matches'] +=1                         
  
    
    if counter >0:
        arrayno.append(counter)
        for key in dict_phrases:
            if dict_phrases[key] >0:
                arrayno.append(key)
                arrayno.append(dict_phrases[key])
                
                
        for i in range(1,len(arrayno)):
            if i %2==0:
                arrayno[i]= '{:3.2f}'.format(arrayno[i]/counter*100)
       
        return arrayno
    else:
        arrayno.append("No Matches")
        return arrayno
   

def phrase_display():
    greeting = phrase_generator()
    
    greeting_display= tk.Text(master=window,height =1, width=100,font=("Arial", 10))
    total = max(dict_phrases['Smart Economy'] + \
            dict_phrases['Smart Mobility'] + \
            dict_phrases['Smart Environment'] + \
            dict_phrases['Smart Governance'] + \
            dict_phrases['Smart People']+  \
            dict_phrases['Smart Living'],1)

    Economy_per = "{:.0%}".format(dict_phrases['Smart Economy']/total)
    Mobility_per = "{:.0%}".format(dict_phrases['Smart Mobility']/total)
    Environment_per = "{:.0%}".format(dict_phrases['Smart Environment']/total)
    Governance_per = "{:.0%}".format(dict_phrases['Smart Governance']/total)
    People_per = "{:.0%}".format(dict_phrases['Smart People']/total)
    Living_per = "{:.0%}".format(dict_phrases['Smart Living']/total)
    
    messagebox.showinfo("       Results of mapping", "Results:  \n  \
                                Smart Economy: \t \t {} \t {} \n  \
                                Smart Mobility: \t \t {} \t {} \n  \
                                Smart Environment: \t {} \t {} \n  \
                                Smart Governance: \t {} \t {} \n  \
                                Smart People: \t \t {} \t {} \n  \
                                Smart Living: \t \t {} \t {} \n ".
                                format(dict_phrases['Smart Economy'], Economy_per, 
                                        dict_phrases['Smart Mobility'], Mobility_per, 
                                        dict_phrases['Smart Environment'], Environment_per,
                                        dict_phrases['Smart Governance'], Governance_per, 
                                        dict_phrases['Smart People'],People_per,
                                        dict_phrases['Smart Living'], Living_per
                                        
                                        ))
    total = 1
    dict_phrases['Smart Economy'] = 0
    dict_phrases['Smart Mobility'] =0
    dict_phrases['Smart Environment'] =0
    dict_phrases['Smart Governance'] =0
    dict_phrases['Smart People'] = 0
    dict_phrases['Smart Living'] = 0   

button1 = tk.Button(text="Results ", command=phrase_display)
button1.grid(column=1, row=4)
window.mainloop()
